In [2]:
!pip install opencv-python

In [3]:
!pip install tensorflow==2.9.1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.1 (from versions: none)
ERROR: No matching distribution found for tensorflow==2.9.1


In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf

In [4]:
print(tf.__version__)

2.9.1


In [5]:
SAVED_MODEL_PATH = "./savedgraphmodel/saved_model"
IMAGE_PATH = "./test.jpg"

In [6]:
imported = tf.saved_model.load(SAVED_MODEL_PATH)
imported

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1ab5c0fe3e0>

In [7]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)

In [8]:
f = imported.signatures['serving_default'] #f = fungsi predict
detections = f(input_tensor)
detections['detection_classes'] = detections['detection_classes'].numpy().astype(np.int64)[0]
detections['detection_scores'] = detections['detection_scores'][0].numpy()

In [9]:
# detection_classes should be ints.
label_id_offset = 1
image_np_with_detections = image_np.copy()

print(detections['detection_classes'])
print(detections['detection_scores'])

[7 3 5 1 7 2 6 5 5 3 1 1 2 2 5 7 1 5 1 1 1 5 3 5 6 5 2 2 5 6 7 1 2 2 1 1 3
 2 2 5 4 7 5 5 5 6 6 1 7 2 3 7 2 2 2 2 2 2 6 2 1 1 1 1 1 5 1 6 5 4 1 7 7 1
 1 7 7 5 1 7 2 1 7 6 1 7 3 3 2 7 3 3 1 1 1 1 2 1 6 7]
[0.73897487 0.7105948  0.12931722 0.10646635 0.08671424 0.08250029
 0.0822864  0.08064631 0.07947291 0.07557546 0.06484652 0.055234
 0.0513732  0.04316761 0.04314344 0.0385309  0.0379172  0.03729143
 0.0365388  0.03366179 0.03276235 0.03090406 0.03021266 0.03009553
 0.02994499 0.02985834 0.02721176 0.02622986 0.02584774 0.0250982
 0.02475058 0.0245643  0.02405693 0.02226906 0.02197085 0.02171806
 0.02166956 0.021595   0.02141851 0.02080331 0.02059086 0.02028325
 0.01976187 0.01963297 0.01927727 0.01899751 0.01804146 0.01778022
 0.0176042  0.01743761 0.0173619  0.01727159 0.01712015 0.01704152
 0.01645095 0.01636379 0.01630968 0.01597894 0.01581355 0.01543751
 0.01510213 0.01491098 0.01461031 0.0145761  0.01456442 0.01456116
 0.01427236 0.0140716  0.01404951 0.0139633  0.01393502 0.0136

In [10]:
dict = {}
for det_class in detections['detection_classes']:
    if (det_class not in dict):
        dict[det_class] = 1
    dict[det_class] += 1

print(dict)
print(len(detections['detection_classes']))
# Most confident prediction (will be used as the prediction result)
print(detections['detection_scores'].max())
print(len(detections['detection_scores']))

{7: 17, 3: 10, 5: 17, 1: 29, 2: 21, 6: 10, 4: 3}
100
0.73897487
100


In [11]:
def get_results(detections, threshold):
  last_valid_idx = 0
  while (detections['detection_scores'][last_valid_idx] >= threshold):
    last_valid_idx += 1
  
  used_scores = detections['detection_scores'][:last_valid_idx]
  used_classes = detections['detection_classes'][:last_valid_idx]
  return used_scores, used_classes

In [28]:
def predict(threshold):
    threshold = float(threshold)
    #get image by http-get
    image_from_api = IMAGE_PATH
    img = cv2.imread(image_from_api)
    images_np = np.array(img)

    # change file image into numpy array
    input_tensor = tf.convert_to_tensor(np.expand_dims(images_np, 0), dtype=tf.uint8)

    detections = f(input_tensor)
    detections['detection_classes'] = detections['detection_classes'].numpy().astype(np.int64)[0]
    detections['detection_scores'] = detections['detection_scores'][0].numpy()

    used_scores, used_classes = get_results(detections, threshold)

    decoded_classes = []
    for i in range (len(used_classes)):
      decoded_classes.append(label_mapping['{}'.format(used_classes[i])])

    
    return decoded_classes


In [17]:
label_mapping = {
    '1': 'kantong',
    '2': 'kertas',
    '3': 'piring',
    '4': 'kardus',
    '5': 'cup',
    '6': 'kaleng',
    '7': 'botol'
}

In [29]:
predict(0.7)

['botol', 'piring']